1. 讀取 GPS 資料(原始)

In [ ]:
import pandas as pd
GPS_Data = pd.read_csv('******',encoding = 'utf-8')
# 取起點與終點
GPS_Data = GPS_Data.head(270).tail(100)
print(GPS_Data)
print(GPS_Data.columns)
lanlot = GPS_Data[['lon','lat']]
print(lanlot.iat[1,0])

2. 畫出圖來並標記於地圖

In [ ]:
import folium
m = folium.Map((lanlot.iat[1,1],lanlot.iat[1,0]),zoom_start=18)
for i in range(len(lanlot.index)):
    folium.CircleMarker(location=(lanlot.iat[i,1],lanlot.iat[i,0]),radius=2,color = "blue").add_to(m)
m

3. 找出兩點之間距離與時間差

In [ ]:
# 建立dataframe
import datetime
import math
import matplotlib.pyplot as plt
import seaborn as sns
lon = GPS_Data['lon']
lat = GPS_Data['lat']
Time = GPS_Data['time']
FMT = '%Y-%m-%d %H:%M:%S'
TimeDiff = []; lonDiff = []; latDiff = []; distance = []; vector = []

#for i in range(1,len(Time)):
for i in range(171,270):
    # 時間差計算
    s = datetime.datetime.strptime(Time[i], FMT) - datetime.datetime.strptime(Time[i-1], FMT)
    TimeDiff.append(int(s.total_seconds()))
    # 經度差計算
    r = lon[i] - lon[i-1]
    lonDiff.append(r)
    # 緯度差計算
    k = lat[i] - lat[i-1]
    latDiff.append(k)
    # 距離計算
    c = math.sqrt(r**2+k**2)
    distance.append(c)
    # 速度計算
    if (int(s.total_seconds()) == 0):
        vector.append(0)
    else:
        v = c/int(s.total_seconds())
        vector.append(v)
plt.hist(vector,color='g',alpha = 0.75)
plt.close()
figbox = plt.figure()
print(vector)
sns.boxplot(x=vector)
figbox.savefig('boxplotEX.png')

4.1 使用廣義線性模型進行點平滑化

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
Predict_lat = []
for i in range(1,21):
    GPS_DataR = GPS_Data.head(5*i).tail(5)
    model_GLM = smf.glm(formula = "lat ~ lon",data = GPS_DataR,family = sm.families.Gaussian())
    # fit the model
    result_GLM = model_GLM.fit()
    # display the result
    result_GLM.summary()
    # prediction
    prediction_GLM = result_GLM.predict()
    #Predict_lat = prediction_GLM
    for j in range(5):
       Predict_lat.append(prediction_GLM[j])
#figGLM = plt.figure()
plt.plot(lon,lat,color="b")
#print(Predict_lat[0])
plt.plot(lon,Predict_lat,  'g')
#figGLM.savefig('glmOr.png')

4.2 使用區域性回歸模型(local regression)進行點平滑化

In [ ]:
import statsmodels.api as sm
x = lat
y = lon
model_lowess = sm.nonparametric.lowess(x,y, frac= 0.05 , it=3)
fig = plt.figure()
plt.plot(lon,lat,color="b")
plt.plot(model_lowess[:,0],model_lowess[:,1],  '-r')
fig.savefig('loessEX.png')

5. 畫出平滑化後的點並標記在地圖

In [ ]:
import folium
n = folium.Map((model_lowess[0,1],lanlot.iat[1,0]),zoom_start=18)
for i in range(len(lanlot.index)):
    # 原資料點
    folium.CircleMarker(location=(lanlot.iat[i,1],lanlot.iat[i,0]),radius=2,color = "blue").add_to(n)
    # loess 資料點
    folium.CircleMarker(location=(model_lowess[i,1],model_lowess[i,0]),radius=2,color = "red").add_to(n)
    # 分段 glm 資料點
    folium.CircleMarker(location=(Predict_lat[i],lanlot.iat[i,0]),radius=2,color = "green").add_to(n)
n